In [1]:
import heapq
import numpy as np
import copy

In [2]:
class Individual:
    def __init__(self, nodes, weight):
        self.nodes = set(nodes)
        self.weight = weight

    def __str__(self):
        return f"{self.nodes}, weight: {self.weight}"

    def __repr__(self):
        return f"{self.nodes}, weight: {self.weight}"

    def __eq__(self, other):
        return self.nodes == other.nodes

    def __lt__(self, other):
        return self.weight < other.weight

    def __hash__(self):
        return hash(tuple(self.nodes))


In [3]:
class Population:
    def __init__(self, individuals):
        self.individuals = individuals
        self.weights = np.zeros(len(individuals))
        self.total_weight = 0
        for i, individual in enumerate(individuals):
            self.weights[i] = individual.weight
            self.total_weight += individual.weight

    def add_individual(self, individual):
        self.individuals.append(individual)
        self.total_weight += individual.weight
        self.weights = np.append(self.weights, individual.weight)

    def size(self):
        return len(self.individuals)

    def __repr__(self):
        return "\n".join([str(individual) for individual in self.individuals])

    def __getitem__(self, item):
        return self.individuals[item]

    def get_best(self, n):

        unique_sequences = set()
        individuals = []
        indexes = []


        for i, individual in enumerate(self.individuals):
            if tuple(individual.nodes) not in unique_sequences:
                unique_sequences.add(tuple(individual.nodes))
                individuals.append(individual)
                indexes.append(i)

        weights = [ind.weight for ind in individuals]
        best_idx = np.argsort(weights)[-n:]

        return [individuals[i] for i in best_idx]

In [4]:
def create_individual(matrix: np.ndarray, weights: np.ndarray):

    result = []
    indexes = np.arange(0, matrix.shape[0], dtype=int)

    while indexes.size > 0:
        idx = np.random.randint(0, indexes.size)
        result.append(indexes[idx])

        tmp = np.where(matrix[idx] == 1)
        indexes = np.delete(indexes, tmp, 0)
        matrix = np.delete(np.delete(matrix, tmp, 0), tmp, 1)

    return Individual(sorted(result), np.sum(weights[result]))


def create_population(matrix: np.ndarray, weights: np.ndarray, population_size=20):

    return Population([create_individual(matrix, weights) for _ in range(population_size)])

In [5]:
def selection_by_weight(population: Population):

    weights = population.weights

    indexes = np.arange(0, population.size(), dtype=int)
    np.random.shuffle(indexes)

    result = []

    for i in range(indexes.size // 2):
        result.append(population[i] if weights[i] > weights[-i - 1] else population[-i - 1])

    return Population(result)

In [6]:
def parent_selection_out(population: Population, skip_p=0.):

    result = []
    first_parent_indexes = np.random.randint(0, population.size(), population.size() // 2)

    for first_parent_index in first_parent_indexes:

        max_diff = 0
        max_diff_idx = 0

        for second_parent_index in range(population.size()):
            if first_parent_index == second_parent_index:
                continue

            if np.random.random() < skip_p:
                continue

            diff = len(population[first_parent_index].nodes.symmetric_difference(population[second_parent_index].nodes))

            if diff > max_diff:
                max_diff = diff
                max_diff_idx = second_parent_index

        result.append((population[first_parent_index], population[max_diff_idx]))

    return result

In [238]:
def truncation_greedy(matrix, weights):

    matrix = np.copy(matrix)

    if matrix[0, 0]:
        matrix -= np.eye(matrix.shape[0], dtype=int)

    deleted_indexes = []

    while matrix.sum() > 0:

        current_node = np.argmax(matrix.sum(axis=0) / (weights + 1e-5))
        matrix[current_node] = 0
        matrix[:, current_node] = 0
        deleted_indexes.append(current_node)

    return set(range(matrix.shape[0])).difference(set(deleted_indexes))


def truncation_random(matrix, weights, temperature=1., alpha=1.):

    matrix = np.copy(matrix)

    if matrix[0, 0]:
        matrix -= np.eye(matrix.shape[0], dtype=int)

    deleted_indexes = []

    indexes = np.arange(0, matrix.shape[0], dtype=int)

    while matrix.sum() > 0:

        coefs = matrix.sum(0) - alpha * weights
        p = np.exp(coefs / temperature)
        p /= np.sum(p)

        current_node = np.random.choice(indexes, p=p)
        matrix[current_node] = 0
        matrix[:, current_node] = 0
        deleted_indexes.append(current_node)

    return set(indexes).difference(set(deleted_indexes))


def expansion_greedy(indexes, matrix, weights):

    candidates = set(np.arange(0, matrix.shape[0], dtype=int))
    indexes_to_delete = set()

    for idx in indexes:
        indexes_to_delete.update(set(*np.where(matrix[idx] == 1)))

    candidates = np.array(list(candidates.difference(indexes_to_delete)))

    while len(candidates) > 0:
        sub_matrix = matrix[candidates]
        sub_matrix = sub_matrix[:, candidates]
        sub_weights = weights[candidates]
        current_node = np.argmin(sub_matrix.sum(axis=0) / (sub_weights + 1e-5))
        indexes.append(candidates[current_node])
        candidates = np.delete(candidates, np.where(sub_matrix[current_node] == 1), 0)

    return indexes


def expansion_random(indexes, matrix, weights, temperature, alpha):

    candidates = set(np.arange(0, matrix.shape[0], dtype=int))
    indexes_to_delete = set()

    for idx in indexes:
        indexes_to_delete.update(set(*np.where(matrix[idx] == 1)))

    candidates = np.array(list(candidates.difference(indexes_to_delete)))

    while len(candidates) > 0:
        sub_matrix = matrix[candidates]
        sub_matrix = sub_matrix[:, candidates]
        sub_weights = weights[candidates]

        coefs = alpha * sub_weights - sub_matrix.sum(axis=0)
        p = np.exp(coefs / temperature)
        p /= np.sum(p)

        current_node = np.random.choice(np.arange(0, sub_matrix.shape[0], dtype=int), p=p)
        indexes.append(candidates[current_node])
        candidates = np.delete(candidates, np.where(sub_matrix[current_node] == 1), 0)

    return indexes


def mutation(nodes_list, max_node_idx):
    node_to_add = np.random.choice(list(set(range(max_node_idx)).difference(set(nodes_list))), replace=False)
    node_to_delete = np.random.randint(0, len(nodes_list), np.random.randint(0, 5))

    nodes_list = list(np.delete(nodes_list, node_to_delete, 0))
    nodes_list.append(node_to_add)

    return nodes_list


def breed(lhs: Individual, rhs: Individual, matrix: np.ndarray, weights: np.ndarray, mutation_p=0.1):

    all_nodes = list(lhs.nodes.union(rhs.nodes))

    if np.random.random() < mutation_p:
        all_nodes = mutation(all_nodes, matrix.shape[0])

    sub_matrix = matrix[all_nodes]
    sub_matrix = sub_matrix[:, all_nodes]

    sub_weights = weights[all_nodes]

    truncated = np.array(all_nodes)[list(truncation_random(sub_matrix, sub_weights, 2., 0.5))]

    expanded_nodes = expansion_random(list(truncated), matrix, weights, 2., 0.5)

    return Individual(expanded_nodes, np.sum(weights[expanded_nodes]))


def breed_all_parents(parents, matrix, weights, mutation_p=0.1):

    result = []
    for lhs, rhs in parents:
        result.append(breed(lhs, rhs, matrix, weights, mutation_p))

    return result

In [239]:
def genetic_algorithm(matrix, weights, population_size=20, keep_best=5, mutation_p=0.1, skip_parent_p=0.):

    population = create_population(matrix, weights, population_size)

    no_change = 0

    unique_bodies = set()
    top_scores_heap = []

    while no_change < 10:
        selected_population = selection_by_weight(population)
        parents = parent_selection_out(population, skip_parent_p)
        children = breed_all_parents(parents, matrix, weights, mutation_p)

        for child in children:
            selected_population.add_individual(child)

        population = selected_population

        previous_top_three = copy.deepcopy(top_scores_heap)

        for individual in population.individuals:
            if tuple(individual.nodes) not in unique_bodies:
                unique_bodies.add(tuple(individual.nodes))
                if len(top_scores_heap) < keep_best:
                    heapq.heappush(top_scores_heap, individual)
                else:
                    heapq.heappushpop(top_scores_heap, individual)

        top_scores_heap.sort(reverse=False)

        if previous_top_three == top_scores_heap:
            no_change += 1
        else:
            no_change = 0

    return top_scores_heap


In [273]:
compatibility_matrix = np.array([
    [0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1],
    [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
])

adjacency_matrix = 1 - compatibility_matrix - compatibility_matrix.T
weights = np.array([6.7871, 0.4965, 0.5813, 6.3638, 7.7534, 6.7388, 2.1945, 5.7203, 2.5396, 0.039, 12.3052, 5.3397, 0.7031, 4.5140, 1.4971, 5.8530, 1.8569, 3.5543, 1.9219,11.8509, 2.2925, 5.1243, 9.2748, 2.9068, 5.4177, 3.3817, 3.5976, 0.5599])

In [278]:
genetic_algorithm(adjacency_matrix, weights, 10, keep_best=5, mutation_p=0.4, skip_parent_p=0.)

[{0, 3, 4, 5, 10, 11, 12, 13, 17, 18, 19, 22, 24, 25, 26}, weight: 89.50399999999999,
 {0, 4, 5, 7, 9, 10, 11, 12, 13, 16, 18, 19, 21, 22, 24, 26, 27}, weight: 89.5046,
 {0, 3, 4, 5, 9, 10, 11, 12, 13, 18, 19, 20, 21, 22, 24, 25, 27}, weight: 90.36779999999999,
 {0, 3, 4, 5, 9, 10, 11, 12, 16, 18, 19, 21, 22, 23, 24, 25, 26, 27}, weight: 91.92259999999999,
 {0, 3, 4, 5, 9, 10, 11, 12, 13, 16, 18, 19, 21, 22, 24, 25, 26, 27}, weight: 93.5298]

In [271]:
%%timeit

genetic_algorithm(adjacency_matrix, weights, 10, keep_best=5, mutation_p=0.4, skip_parent_p=0.)

81.2 ms ± 5.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [158]:
adjacency_matrix = np.array([
    [0, 1, 0, 0, 0, 0, 1, 1],
    [1, 0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 1, 0],
    [0, 0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0],
])
tmp = np.arange(0, adjacency_matrix.shape[0], dtype=int)
adjacency_matrix[tmp, tmp] = 1
adjacency_matrix

array([[1, 1, 0, 0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0],
       [0, 1, 0, 0, 1, 1, 0, 0],
       [1, 1, 1, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 1]])

In [159]:
weights = np.array([1.2, 2, 4, 2, 1.5, 1.5, 8.9, 5])

In [214]:
truncated = truncation_random(matrix=adjacency_matrix, weights=weights, temperature=2., alpha=0.5)
truncated

{3}

In [237]:
expanded_nodes = expansion_random(list(truncated), adjacency_matrix, weights, 3, 0.5)
expanded_nodes

[3, 2, 0, 5]